In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from IPython import display

# The neuron non-linearity

In [ ]:
def ReLU(z):
    return np.maximum(0,z)
    
def ReLUPrime(z):
    return 1 if z > 0 else 0

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def sigmoidPrime(z):
    # Derivative of Sigmoid Function
    # dy/dz = e^-z / (1+e^-z)^2 = y * (1-y)
    return np.exp(-z) / ((1+np.exp(-z))**2)

# apply the function on every element of a vector
ReLU = np.vectorize(ReLU)
ReLUPrime = np.vectorize(ReLUPrime)
sigmoid = np.vectorize(sigmoid)
sigmoidPrime = np.vectorize(sigmoidPrime)

f = ReLU
fPrime = ReLUPrime

In [ ]:
def sample():
    x = np.random.uniform(low=0.0, high=50.0)
    y = np.random.uniform(low=0.0, high=30.0)
    z_1 = 0.25 * x + 0.75 * y + np.random.normal(loc=0.0, scale=0.01)
    z_2 = .33333 * y + .66666 * x + np.random.normal(loc=0.0, scale=0.01)
    return np.array([x, z_1, y, z_2])

In [ ]:
def error(target, output):
    return .5 * np.sum(map(lambda (t, y): (t-y)**2, zip(target, output)))

## Layer containing neurons and weights from prev layer

In [ ]:
class Layer(object):
    def __init__(self, name, N_in, N, norm=True, weight_scale=0.01, with_bias=False):
        self.name = name
        self.N = N
        self.N_in = N_in
        
        # summed input
        self.Z = np.zeros(N)
        self.E_Z = np.zeros(N)
        
        # and applied non-linearity
        self.Y = np.zeros(N)
        self.E_Y = np.zeros(N)
        
        # incomming weights
        bias = 1 if with_bias else 0
        #self.W = np.ones((N_in + bias, N)) * 0.25
        if norm:
            self.W = np.random.normal(loc=0.0, scale=weight_scale, size=(N_in + bias, N))
        else:
            self.W = np.random.uniform(low=0.0, high=weight_scale, size=(N_in + bias, N))
        self.E_W = np.zeros(self.W.shape)
        self.V = np.zeros(self.W.shape)
        
    def reset_error(self):
        self.E_Y *= 0
        self.E_Z *= 0
        self.E_W *= 0
    
    def write(self):
        print"Name: ", self.name
        print"Y   :\n", self.Y
        print"E_Y :\n", self.E_Y
        print"Z   :\n", self.Z
        print"E_Z :\n", self.E_Z
        print"W   :\n", self.W
        print"E_W :\n", self.E_W

    def plot(self):
        f, axes = plt.subplots(2, 1)
        (axW, axY) = axes
        axW.imshow(self.W,
                   cmap="gray",
                   interpolation='none',
                   vmax=np.max(self.W), vmin=np.min(self.W))
        axY.imshow([self.Y],
                   cmap="gray",
                   interpolation='none',
                   vmax=np.max(self.Y), vmin=np.min(self.Y))
        for a in axes:
            a.get_xaxis().set_visible(False)
            a.get_yaxis().set_visible(False)

## Forward Path

In [ ]:
def forward(layers, training, with_bias=False):
    """
    layers np.array with multiple layers
    training np.array with one training case
    """
    I = training
    if with_bias:
        I = np.append(I, [1.0])
    for l in layers:
        l.Z = I.dot(l.W)
        l.Y = f(l.Z)
        I = l.Y
        if with_bias:
            I = np.append(I, [1.0])
    
    return I[0:len(layers[-1].Y)]

## Backward Path and Error Computing

In [ ]:
def backward(layers, training, with_bias=False):
    """
    layers np.array with multiple layers
    training np.array with one training case
    """
    output = layers[1]
    hidden = layers[0]
    # Output layer
    # dE/dY_O = (-(target - output)) = output - target
    output.E_Y = output.Y - training
    # dE/dZ_O = dE/dY_O * dY_O/dZ_O
    output.E_Z = output.E_Y * fPrime(output.Z)
    # dE/dW2 = Y_H * dE/dZ_O
    # mini-batch
    Y = hidden.Y
    if with_bias:
        Y = np.append(Y, [1.0])
    output.E_W += np.transpose([Y]) * [output.E_Z]
    
    # Hidden layer
    # dE/dY_H = dE/dZ_O * trans(W2)
    hidden.E_Y = output.E_Z.dot(np.transpose(output.W))
    # dE/dZ_H = dE/dY_H * dY_H/dZ_H
    hidden.E_Z = hidden.E_Y[0:len(hidden.Z)] * fPrime(hidden.Z)
    # dE/dW1 = Y_I * dE/dZ_H
    # mini-batch
    Y = training
    if with_bias:
        Y = np.append(Y, [1.0])
    hidden.E_W += np.transpose([Y]) * [hidden.E_Z]

## Momentum SGD Learning

In [ ]:
def learn(layers, eps, batch_size, momentum):
    for l in layers:
        V_curr = momentum * l.V - eps * l.E_W / batch_size
        l.W = l.W + V_curr
        l.V = V_curr

## Batch Learn

In [ ]:
def batch(layers, targets, eps, momentum, with_bias=False):
    err = 0.0
    for target in targets:
        output = forward( layers, target, with_bias )
        backward( layers, target, with_bias )
        err += error(target, output)
    
    learn(layers, eps, len(targets), momentum)
    
    for l in layers:
        l.reset_error()

    return err / len(targets)

## Nesterov's Accelerated Gradient

In [ ]:
def NAG_batch(layers, targets, eps, momentum, with_bias=False):
    # first jump into prev. direction
    V_curr = []
    for l in layers:
        V_tmp = momentum * l.V
        l.W = l.W + V_tmp
        V_curr.append(V_tmp)

    # measure gradients
    err = 0.0
    for target in targets:
        output = forward( layers, target, with_bias )
        backward( layers, target, with_bias )
        err += error(target, output)
    
    # apply gradient
    for l, v in zip(layers, V_curr):
        G_tmp = eps * l.E_W / len(targets)
        l.W = l.W - G_tmp
        l.V = v - G_tmp
    
    for l in layers:
        l.reset_error()

    return err / len(targets)

# Create AutoEncoder

In [ ]:
scale = 0.01
norm = False
use_bias = True
h = Layer("hidden", 4, 2, norm, scale, use_bias)
o = Layer("output", 2, 4, norm, scale, use_bias)
layers = [h,o]

# Learning
learn_errors = []

## Train AutoEncoder

In [ ]:
eps = 0.00001
momentum = 0.99

try:
    for i in xrange(0, 10000):
        if i % 19 == 0:
            plt.clf()
            plt.plot(xrange(0, len(learn_errors)),
                     learn_errors, color='blue')
            plt.yscale('log')
            if i > 100:
                plt.axhline(y=np.mean(learn_errors[-100:-1]), color="red")
            display.display(plt.gcf())
            display.clear_output(wait=True)

        targets = [ sample() for i in xrange(0,100) ]
        err = NAG_batch(layers, targets, eps=eps, momentum=momentum, with_bias=use_bias)
        #err = batch(layers, targets, eps=eps, momentum=momentum, with_bias=use_bias)

        learn_errors += [err]
        if np.mean(learn_errors[-100:-1]) < 1.e-10:
            break
except KeyboardInterrupt:
    print "Interrupted"

## Print

In [ ]:
t = sample()
o = forward(layers, t, use_bias)
for l in reversed(layers):
    l.reset_error()
    l.write()
    print ""
print "Input :", t
print "Output:", o
print "Diff  :", o - t
print "Error :", error(t, o)

In [ ]:
plt.imshow([t],
        cmap="gray",
        interpolation='none',
        vmax=np.max(t), vmin=np.min(t))
plt.axes().get_xaxis().set_visible(False)
plt.axes().get_yaxis().set_visible(False)
for l in layers:
    l.reset_error()
    l.plot()
    print ""

## Validating

In [ ]:
validate_errors = []
for i in xrange(0, 1000):
    target = sample()
    output = forward( layers, target, use_bias )
    err = error(target, output) 
    validate_errors += [err]

plt.plot(xrange(0, len(validate_errors)), validate_errors)
plt.axhline(y=np.mean(validate_errors), color="red")
plt.yscale('log')